# Gather data from the web
As a data analyst, data is the fuel that drives the engine of discovery.  Often times this means using data we have collected with web forms, using external systems or via some business process.  Occassionally, we'll be given the data directly or we can download some data from a webpage that meets our needs.  Being able to gather data from sources on the web is a skill that is often very useful.  Wouldn't it be great if you could download [weather data predictions](https://www.abstractapi.com/api/weather#:~:text=What%20does%20the%20Weather%20API%20do%3F%20Abstract%27s%20Weather,and%20forecasted%20weather%20data%20for%20millions%20of%20locations.) as part of your morning routine, or grab [movies reviews](https://www.rottentomatoes.com/) to make plans for the weekend or even get data from the [James Webb Telescope](https://www.nasa.gov/mission_pages/webb/about/index.html) so you can process your own images from outer space!

Well fortunately you have most of the skills you need to do some of these things already.  In this notebook, we'll look at a few ways to get at data on the web, this is not comprehensive, but it should give you a start in the right direction.  One key skill this notebook leaves out is automating a webpage (like filling out a form or hitting a submit button) this automation takes a bit of extra setup and doesn't work well in a Jupyter notebook, but there are tools that can be helpful.  Check out [selenium](https://www.geeksforgeeks.org/selenium-python-tutorial/) and [scrapy](https://docs.scrapy.org/en/latest/intro/tutorial.html) for a couple of approaches that use Python.

## Simple Webscraping with Pandas
Believe it or not, one of the simplest ways to get data from the web is to use `pandas`, the `read_html` function is fantastic and simple.  All we need is to install a couple libraries and then we can gather data from simpler web pages very quickly.

In this first example, I went to [boxofficemojo.com](https://www.boxofficemojo.com/) and I selected a particular page where I saw some data I needed.  In this case, [Top Lifetime Grosses by MPAA Rating for PG-13 movies](https://www.boxofficemojo.com/chart/mpaa_title_lifetime_gross/?by_mpaa=PG-13&ref_=bo_cso_ac).  Looking at this page, I can see that there are about 200 movies on the first page.  This is enough for our purposes.  Next, I captured the URL (the link in the browser) and I use this to open with the `pandas` library.  Be careful, the result of the `read_html` method is a *list* of dataframes, so we'll have to know which one we are after.  In this particular case there is only one data frame, so we can use that one.


In [5]:
# Start by importing a couple of libraries
import pandas as pd
movies = pd.read_html("https://www.boxofficemojo.com/chart/mpaa_title_lifetime_gross/?by_mpaa=PG-13&ref_=bo_cso_ac")
print(len(movies))

1


In [6]:
# Since there is only one dataframe in the list, we choose the first one
movies_df = movies[0]
movies_df

,Title,Rank,Lifetime Gross,Overall Rank,Year
0,Star Wars: Episode VII - The Force Awakens,1,"$936,662,225",1,2015
1,Avengers: Endgame,2,"$858,373,000",2,2019
2,Spider-Man: No Way Home,3,"$804,793,477",3,2021
3,Avatar,4,"$760,507,625",4,2009
4,Black Panther,5,"$700,426,566",5,2018
...,...,...,...,...,...
195,Fantastic Beasts: The Crimes of Grindelwald,196,"$159,555,901",353,2018
196,Star Trek Beyond,197,"$158,848,340",358,2016
197,The Longest Yard,198,"$158,119,460",360,2005
198,X-Men,199,"$157,299,718",361,2000


With one simple command, we have scraped 200 rows of data!  A little clicking [around on the website](https://www.boxofficemojo.com/chart/mpaa_title_lifetime_gross/?by_mpaa=PG-13&offset=200) and I can gather another page of movies as well.

Now, with a little detective work I can see that the only difference between the two URLs is that last item `offset=200`.  I wonder what happens if I change that value to `offset=400`?  Try it, I'll wait here....

In [7]:
# Since I already know there is only 1 dataframe in the list, let's just grab the first one immediately
movies_200_df = pd.read_html("https://www.boxofficemojo.com/chart/mpaa_title_lifetime_gross/?by_mpaa=PG-13&offset=200")[0]
movies_200_df

,Title,Rank,Lifetime Gross,Overall Rank,Year
0,X-Men: Apocalypse,201,"$155,442,489",367,2016
1,The Mummy,202,"$155,385,488",368,1999
2,Snow White and the Huntsman,203,"$155,332,381",369,2012
3,San Andreas,204,"$155,190,832",370,2015
4,Fast & Furious,205,"$155,064,265",371,2009
...,...,...,...,...,...
195,Prince of Persia: The Sands of Time,396,"$90,759,676",868,2010
196,Scary Movie 4,397,"$90,710,620",870,2006
197,Madea Goes to Jail,398,"$90,508,336",875,2009
198,Space Cowboys,399,"$90,464,773",876,2000


You can see where this is going right?  I could easily setup a loop and simply change the value of `offset` from 200 to 400 to 600 to 800 and gather all the top 1000 movies.

In [14]:
# Build the basic url string
base_url = 'https://www.boxofficemojo.com/chart/mpaa_title_lifetime_gross/?by_mpaa=PG-13'

# Start over with our movies dataframe, in case this cell runs more than once
movies_df = pd.read_html(base_url)[0]
# We need a list to put all the dataframes in prior to joining it back together
movie_list = []
# Go through the loop by 200s from 200 to 1000 (step = 200)
for offset in range(200,1000,200):
    # Store 200 movies at a time into a temporary dataframe
    temp_url = base_url+f'&offset={offset}'
    print (temp_url)
    m_df = pd.read_html(temp_url)[0]
    movie_list.append(m_df)

# Now that we have all the movies in a list, append the datasets together into one master dataframe
movies_df = movies_df.append(movie_list,ignore_index=True)
movies_df


https://www.boxofficemojo.com/chart/mpaa_title_lifetime_gross/?by_mpaa=PG-13&offset=200
https://www.boxofficemojo.com/chart/mpaa_title_lifetime_gross/?by_mpaa=PG-13&offset=400
https://www.boxofficemojo.com/chart/mpaa_title_lifetime_gross/?by_mpaa=PG-13&offset=600
https://www.boxofficemojo.com/chart/mpaa_title_lifetime_gross/?by_mpaa=PG-13&offset=800


/tmp/ipykernel_8688/653906905.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  movies_df = movies_df.append(movie_list,ignore_index=True)


,Title,Rank,Lifetime Gross,Overall Rank,Year
0,Star Wars: Episode VII - The Force Awakens,1,"$936,662,225",1,2015
1,Avengers: Endgame,2,"$858,373,000",2,2019
2,Spider-Man: No Way Home,3,"$804,793,477",3,2021
3,Avatar,4,"$760,507,625",4,2009
4,Black Panther,5,"$700,426,566",5,2018
...,...,...,...,...,...
995,Hot Pursuit,996,"$34,580,201",2531,2015
996,Concussion,997,"$34,542,474",2533,2015
997,Malibu's Most Wanted,998,"$34,432,201",2535,2003
998,Like Father Like Son,999,"$34,377,585",2539,1987


Now that is super helpful!  We just downloaded 1000 top grossing pg-13 rated movies in a few seconds.  But what exactly was that thing we did at the end, the whole `offset=200` thing.  This is called a query parameter and be a very transparent way for webpages to communicate to the server about the data they want.

## Query parameters
A bit of background, (a very little bit), web page links usually only "resolve" to a particular page on a particular server.  The webserver looks up the page and returns whatever is stored there.  This isn't very handy if you have alot of data to display - for instance in the case of boxofficemojo.com, they don't want to have to create 5 different web pages for each set of 200 movies.  So instead they built one web page and used parameters (yes!  just like function parameters in Python) which define the exact contents to display.  Fortunately for us, this makes it very transparent how to manipulate the page.  We can see exactly what details the server needs to give the data we want.

The URL format is, `<protocol>` (like https://) followed by the endpoint (www.boxofficemojo.com/chart/mpaa_title_lifetime_gross) then a `?` followed by 0 or more query parameters.  Query parameters are name/value pairs and delimited by an `&`.  So let's look at that url from before again.

> https://www.boxofficemojo.com/chart/mpaa_title_lifetime_gross/?by_mpaa=PG-13&offset=200

Here the query parameters are `by_mpaa` and `offset`.  So that's interesting, now that we notice this, it's possible that we could change the `by_mpaa` parameter as well and get R rated movies as well?  Let's try it by hand first.  Paste the link into the website and change the PG-13 to R and see what happens.

That's great, so with a simple change we could also R rated movies in a dataframe as well.  (You can copy the code above and try it out for yourself below).

In [16]:
# Copy the code from above and replace PG-13 with R to get the top 100 grossing R rated movies

It's kind of a pain to navigate all that string building and trying to keep up with encoding each part of the URL.  Spaces for instance, can't be used in a parameter and what if you need to use an `&` symbol for a parameter name.  Well fortunately, we have a helpful built-in library called `urllib` which takes care to make sure our format is right everytime.  All we need to do is supply a dictionary of parameters and tell the library to encode it into a proper request.

In [22]:
from urllib.parse import urlencode

# Build the basic url string
base_url = 'https://www.boxofficemojo.com/chart/mpaa_title_lifetime_gross/?'

# Setup the parameters
params = {'by_mpaa':'R','offset':0}
print(base_url + urlencode(params))

https://www.boxofficemojo.com/chart/mpaa_title_lifetime_gross/?by_mpaa=R&offset=0


You should now be able to re-write the downloaer using the URL encoder for the query parameters.

## Using APIs
### Query Based Parameters
### REST services

## Simple Webscraping with BeautifulSoup